# Booklist

> Selectable list of Calibre books for Ragfood

In [ ]:
#| default_exp booklist

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json, os, subprocess
from ipywidgets import Button, Box, HBox, VBox, Dropdown, Accordion, Label
from Ragfood.Selectable import Selectable
from Ragfood.mediator import Colleague
from Ragfood.states import *

In [ ]:
#| export
class BooklistItem(Selectable):
    """Individual book item in the booklist."""
    
    def setInitState(self):
        """Set title and embedding-icon"""
        self.item = Label()
        self.refresh()
        self.setItemWidget(self.item)
    
    def refresh(self):
        """Refresh the display with current book info."""
        icon = '📜' if '*embedding' in self.book else ''
        self.item.value = f"{icon}{self.book['title']}"
    
    def __init__(self, book, books, selector=None, behave='radio'):
        """Initialize the BooklistItem with given parameters."""
        # attributes
        self.book = book
        
        # states
        super().__init__(books, behave, selector)

In [ ]:
#| export
class Booklist(Colleague):
    """Provides a selectable list of calibre books from the given library."""
    
    def __init__(self, books, heightListArea=600, layout={'width': '300px', 'border': '1px solid', 'height': '100%'}, mediator=None, behave='multi'):
        """Initialize the Booklist widget."""
        # attributes and super constructor
        self.books, self.selections, self.heightListArea, self.widget = books, [], heightListArea, VBox(layout=layout)
        super().__init__(mediator)
        
        # widgets
        self.list_inner_vb = VBox(layout={'height': f'{self.heightListArea}px', 'width': '100%', 'overflow': 'auto'})
        self.list_outer_bx = Box(children=[self.list_inner_vb], layout={'width': '100%', 'overflow': 'auto'})
        self.widget.children = [self.list_outer_bx]
        
        # create items for ids
        self.booklistWidgets = []
        self.booklistItems = [BooklistItem(book=book,
                                          books=self.booklistWidgets,
                                          selector=self.selector,
                                          behave=behave) for book in self.books]
        self.list_inner_vb.children = [booklistItem.widget for booklistItem in self.booklistItems]
    
    def selector(self, selections, lastSelect):
        """Handle book selection events."""
        self.selections, self.lastSelect = selections, lastSelect
        self.changed('OnBooklistBookSelected',
                    RAGFOOD_OK,
                    positon_list=selections,
                    last_select=lastSelect,
                    calibre_id=self.books[lastSelect]['id'])
    
    def getSelectedBooks(self):
        """Get list of selected book IDs."""
        return [self.booklistItems[s].book['id'] for s in self.selections]

## Example Usage

In [ ]:
# Example usage (requires Calibre setup)
# from Ragfood.calibre import Calibre
# from Ragfood.mediator import Mediator
# 
# class MiniMedi(Mediator):
#     def notify(self, colleague, event, state, *argc, **argv):
#         print(event, state, argc, argv)
# 
# calibre = Calibre()
# booklist = Booklist(books=calibre.book_list, mediator=MiniMedi(), heightListArea=200)
# display(booklist.widget)